Exercise

In [9]:
import requests
import pandas as pd
from datetime import datetime

In [10]:
baseurl = f"http://api.weatherapi.com/v1/"

In [30]:
weather_data = []
def get_city_data(city, api_key):
    # get current weather
    current_url = f"{baseurl}current.json?key={api_key}&q={city}"               # current API method
    current_response = requests.get(current_url)
    if current_response.status_code == 200:
        current_data = current_response.json()
        location = current_data['location']
        current = current_data['current']
        # build a summary for the city, save that summary into a dictionary
        weather_data.append({
            "City" : city,
            "Temperature" : current['temp_f'],
            "Condition" : current['condition']['text'],
        })
        print(f"Weather for {location['name']}, {location['region']}")
        print(f"Temperature: {current['temp_f']} F / {current['temp_c']} C")
        print(f"Condition: {current['condition']['text']}")
    else:
        print(f"Error fetching data for {city}: {current_response.status_code}")
    
    # get a forecast    
    forecast_url = f"{baseurl}forecast.json?key={api_key}&q={city}&days=3"      # forecast API method
    forecast_response = requests.get(forecast_url)
    if forecast_response.status_code == 200:
        forecast_data = forecast_response.json()
        forecast = forecast_data['forecast']['forecastday']
        forecast_3_days_summary = []
        for day in forecast:
            date_obj = datetime.strptime(day['date'], '%Y-%m-%d')
            formatted_date = date_obj.strftime('%A, %B %d')
            forecast_3_days_summary.append(f"{formatted_date}: High {day['day']['maxtemp_f']}F / Low {day['day']['mintemp_f']}F - {day['day']['condition']['text']}")
    else:
        print(f"Error fetching data for {city}: {forecast_response.status_code}")                

    print(forecast_3_days_summary)

    # get some astronomical data
    today = pd.Timestamp.now().strftime("%Y-%m-%d")
    astro_url = f"{baseurl}astronomy.json?key={api_key}&q={city}&dt={today}"    # astronomy API method
    astro_response = requests.get(astro_url)
    if astro_response.status_code == 200:
        astronomy_data = astro_response.json()
        astronomy = astronomy_data['astronomy']['astro']
        weather_data.append({
            "Sunrise" : astronomy['sunrise'],
            "Sunset" : astronomy['sunset'],
            "Moon Phase" : astronomy['moon_phase'],
        })
        print(f"Sunrise {astronomy['sunrise']} / Sunset {astronomy['sunset']} / Moon Phase: {astronomy['moon_phase']}")
    else:
        print(f"Error fetching data for {city}: {astro_response.status_code}")

    # get local time
    tzone_url = f"{baseurl}timezone.json?key={api_key}&q={city}"               # timezone API method
    tzone_response = requests.get(tzone_url)
    if tzone_response.status_code == 200:
        tzone_data = tzone_response.json()
        localtime = tzone_data['location']
        weather_data.append({
            "Local Time" : localtime['localtime'],
        })
        print(f"Time Zone for {location['name']}, {localtime['localtime']}")
    else:
        print(f"Error fetching data for {city}: {current_response.status_code}")

In [31]:
# call the function
get_city_data("Louisville", "36bb2a64547343d2a68154527250409")

Weather for Louisville, Kentucky
Temperature: 82.0 F / 27.8 C
Condition: Sunny
['Friday, September 05: High 84.0F / Low 58.8F - Partly Cloudy ', 'Saturday, September 06: High 73.0F / Low 59.7F - Patchy rain nearby', 'Sunday, September 07: High 76.6F / Low 53.2F - Sunny']
Sunrise 07:17 AM / Sunset 08:06 PM / Moon Phase: Waxing Gibbous
Time Zone for Louisville, 2025-09-05 13:33


In [32]:
# use the function to build a df that has the city data 
exercise_df = pd.DataFrame(weather_data)

In [33]:
# ...then save it into a csv file
exercise_df.to_csv("../data/exercise.csv", index=False)